## Backward difference coding Using `Categorical encoders`

https://contrib.scikit-learn.org/categorical-encoding/backward_difference.html#

### Installing categorical encoders

In [2]:
! pip install category-encoders

     |████████████████████████████████| 92kB 1.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols

import category_encoders as ce

### Creating a dataframe
The same 'ol Iris dataset

In [4]:
iris_data = pd.read_csv("datasets/iris.csv")

iris_data.sample(10)

,Species,sepal_length,sepal_width,petal_length,petal_width
59,Iris-versicolor,5.2,2.7,3.9,1.4
72,Iris-versicolor,6.3,2.5,4.9,1.5
140,Iris-virginica,6.7,3.1,5.6,2.4
29,Iris-setosa,4.7,3.2,1.6,0.2
105,Iris-virginica,7.6,3.0,6.6,2.1
41,Iris-setosa,4.5,2.3,1.3,0.3
55,Iris-versicolor,5.7,2.8,4.5,1.3
143,Iris-virginica,6.8,3.2,5.9,2.3
62,Iris-versicolor,6.0,2.2,4.0,1.0
48,Iris-setosa,5.3,3.7,1.5,0.2


In [5]:
iris_data.drop(columns=['sepal_length', 'sepal_width', 'petal_width'], 
               inplace=True)

iris_data.sample(5)

,Species,petal_length
122,Iris-virginica,6.7
81,Iris-versicolor,3.7
78,Iris-versicolor,4.5
88,Iris-versicolor,4.1
131,Iris-virginica,6.4


In [6]:
iris_data.describe()

,petal_length
count,150.000000
mean,3.758667
std,1.764420
min,1.000000
25%,1.600000
50%,4.350000
75%,5.100000
max,6.900000


In [7]:
iris_data.groupby(by='Species').mean()

,petal_length
Species,
Iris-setosa,1.464
Iris-versicolor,4.260
Iris-virginica,5.552


In [8]:
iris_data.groupby(by='Species').mean().diff()

,petal_length
Species,
Iris-setosa,NaN
Iris-versicolor,2.796
Iris-virginica,1.292


In [10]:
mod = ols("petal_length ~ C(Species, Diff)", 
          data=iris_data)

res = mod.fit()

res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           petal_length   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     1179.
Date:                Mon, 01 Jul 2019   Prob (F-statistic):           3.05e-91
Time:                        08:18:32   Log-Likelihood:                -84.840
No. Observations:                 150   AIC:                             175.7
Df Residuals:                     147   BIC:                             184.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               3.7587      0.035    106.978      0.000       3.689       3.828
C(Species, Diff)[D.Iris-setosa]         2.7960      0.086     32.488      0.000       2.626       2.966
C(Species, Diff)[D.Iris-versicolor]     1.2920      0.086     15.012      0.000       1.122       1.462
==============================================================================
Omnibus:                        4.393   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.111   Jarque-Bera (JB):                5.370
Skew:                           0.121   Prob(JB):                       0.0682
Kurtosis:                       3.895   Cond. No.                         3.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

https://contrib.scikit-learn.org/categorical-encoding/backward_difference.html

### Creating a object and giving the column which need to be encoded as argument

In [14]:
encoder = ce.BackwardDifferenceEncoder(cols=['Species'])
encoder

BackwardDifferenceEncoder(cols=['Species'], drop_invariant=False,
                          handle_missing='value', handle_unknown='value',
                          mapping=None, return_df=True, verbose=0)

### Dataframe with Encoded columns of  `color`

In [18]:
species_encoded = encoder.fit_transform(iris_data)

species_encoded.head()

,intercept,Species_0,Species_1,petal_length
0,1,-0.666667,-0.333333,1.4
1,1,-0.666667,-0.333333,1.4
2,1,-0.666667,-0.333333,1.3
3,1,-0.666667,-0.333333,1.5
4,1,-0.666667,-0.333333,1.4


In [19]:
species_encoded.sample(5)

,intercept,Species_0,Species_1,petal_length
144,1,0.333333,0.666667,5.7
91,1,0.333333,-0.333333,4.6
50,1,0.333333,-0.333333,4.7
98,1,0.333333,-0.333333,3.0
12,1,-0.666667,-0.333333,1.4


In [20]:
encoded_iris = pd.concat([iris_data['Species'], species_encoded], 
                         axis = 1)

encoded_iris.sample(10)

,Species,intercept,Species_0,Species_1,petal_length
33,Iris-setosa,1,-0.666667,-0.333333,1.4
15,Iris-setosa,1,-0.666667,-0.333333,1.5
75,Iris-versicolor,1,0.333333,-0.333333,4.4
124,Iris-virginica,1,0.333333,0.666667,5.7
102,Iris-virginica,1,0.333333,0.666667,5.9
14,Iris-setosa,1,-0.666667,-0.333333,1.2
69,Iris-versicolor,1,0.333333,-0.333333,3.9
135,Iris-virginica,1,0.333333,0.666667,6.1
73,Iris-versicolor,1,0.333333,-0.333333,4.7
121,Iris-virginica,1,0.333333,0.666667,4.9


In [21]:
X = encoded_iris.drop(columns=['Species', 'petal_length'])

y = encoded_iris.petal_length

In [22]:
X.head()

,intercept,Species_0,Species_1
0,1,-0.666667,-0.333333
1,1,-0.666667,-0.333333
2,1,-0.666667,-0.333333
3,1,-0.666667,-0.333333
4,1,-0.666667,-0.333333


In [23]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression(fit_intercept=False)

linear_model.fit(X, y)

print("Training_score : " , linear_model.score(X, y))

Training_score :  0.9413189735606261


In [24]:
linear_model.coef_

array([3.75866667, 2.796     , 1.292     ])

In [25]:
linear_model.intercept_

0.0